# Title
[]()

In [122]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [119]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [89]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

In [123]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
# from summary_chain import Chaining
# from article_processing import create_text_dict, create_text_dict_from_folder


In [136]:
from wrangling import filter_df_all_conditions
from response_processing import process_chaining_results, save_instance_to_dict

# Create text_dict

In [3]:
folder_path = '../text/2023-04-14'
encoding='ISO-8859-1'
subset=None

text_dict = create_text_dict_from_folder(folder_path, encoding='ISO-8859-1', subset=subset)
text_dict

{1: 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middl

# Create prompts

In [88]:
experiment_num = 1

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

summarize_task = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]

simple_simplify_task = [
    # "Use terms a 12-year-old can understand.",
    "Assume your audience has no science background.",
    "Use a fun tone."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

user_simplify_task = [
    "Use language appropriate for",
    # "Use terms a 12-year-old can understand.",
]

simplify_audience = [
    # "",
    "a lay audience",
    "people without a science background",
]

user_relevance_task = [
    # "Add 1-2 sentences to make this relevant for",
    "Update the summary with 1-2 sentences to make it relevant for",
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
]

relevance_audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]



# `Chaining` class scripts

In [93]:

from itertools import product
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role="You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        chatbot = Chaining(self.text)
        full_task = f'{prep_step} {task}'
        prompt = chatbot.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['task'] = task
            self.summaries_dict['prompt'] = full_task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance statement': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict

# Load previous chatbot attributes

In [3]:
filename = 'batch_Chaining_attributes_relevance_2023-04-19_1437.sav'
pickle_path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
chatbot_dict = loadpickle(filename, pickle_path)

Time completed: 2023-05-01 14:46:50.922438
Dictionary keys: ['text1_prompt00', 'text1_prompt01', 'text2_prompt00', 'text2_prompt01']


In [4]:
chatbot_dict['text1_prompt00']

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

In [8]:
chatbot_dict['text1_prompt00']['text']

'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middle ag

# Re-create an instance's attributes from a dictionary

In [45]:
chatbot = Chaining(chatbot_dict['text1_prompt00']['text'])

In [46]:
type(chatbot)

__main__.Chaining

In [47]:
chatbot.text

'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middle ag

In [48]:
vars(chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

In [49]:
chatbot.text2 = 'This is the second text'

In [50]:
vars(chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

In [51]:
setattr(chatbot, 'text3', 'This is the third text')

In [52]:
vars(chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

In [25]:
def revive_chatbot(chatbot):

    print(f'Original chatbot attributes:')
    new_chatbot = Chaining(chatbot['text'])
    for attribute in chatbot.keys():
        print(f'\t{attribute}: {chatbot[attribute]}')
        new_chatbot.attribute = chatbot[attribute]

    

    return new_chatbot

new_chatbot = revive_chatbot(chatbot_dict['text1_prompt00'])

Original chatbot attributes:
	text: Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men

Discussion
Results of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or muscle damage response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (2329 years) and older adults (6065 years) (7,15,24), and between middle-aged (4142 years) and older adults (7072 years) (17,18). It does seem that the decline in strength may develop du

In [33]:
dir(new_chatbot)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_relevance',
 'attribute',
 'create_prompt',
 'gpt',
 'max_tokens',
 'model',
 'simplify',
 'summarize',
 'system_role',
 'temperature',
 'text']

In [32]:
vars(new_chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

## Iteration 1.1

In [61]:
def revive_chatbot(chatbot):

    print(f'Original chatbot attributes:')
    new_chatbot = Chaining(chatbot['text'])
    # for attribute in chatbot.keys():
    #     print(f'\t{chatbot[attribute]}')
    #     # print(f'\t{attribute}: {chatbot[attribute]}')
    #     # setattr(new_chatbot, 'halo', chatbot[attribute])
    #     setattr(new_chatbot, 'halo', 'halll')
    #     print(f'\tNew chatbot attribute added: {new_chatbot.attribute}')

    return new_chatbot

new_chatbot = revive_chatbot(chatbot_dict['text1_prompt00'])

Original chatbot attributes:


In [62]:
vars(new_chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

In [63]:
setattr(new_chatbot, 'halo', 'halll')

In [64]:
vars(new_chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

## 1.2 `revive_chatbot` function iteration 

In [68]:
def revive_chatbot(chatbot):

    new_chatbot = Chaining(chatbot['text'])
    for key in chatbot.keys():
        # print(f'\t{chatbot[key]}')
        setattr(new_chatbot, key, chatbot[key])
        print(f'\tNew chatbot attribute added: {getattr(new_chatbot, key)}')

    return new_chatbot

new_chatbot = revive_chatbot(chatbot_dict['text1_prompt00'])

	New chatbot attribute added: Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men

Discussion
Results of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or muscle damage response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (2329 years) and older adults (6065 years) (7,15,24), and between middle-aged (4142 years) and older adults (7072 years) (17,18). It does seem that the decline in strength may develop during m

In [69]:
vars(new_chatbot)

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

# revive_chatbot for the entire chatbot dictionary

In [79]:
def revive_chatbot(chatbot):

    new_chatbot = Chaining(chatbot['text'])
    print(f'Article title: {chatbot["article_title"]}')
    for key in chatbot.keys():
        # print(f'\t{chatbot[key]}')
        setattr(new_chatbot, key, chatbot[key])
        print(f'\tNew chatbot attribute added: {key}')
        if type(getattr(new_chatbot, key)) == dict:
            print(f'\t\tAttribute dictionary keys: {[attr for attr in getattr(new_chatbot, key)]}')
    return new_chatbot

def revive_chatbot_dict(chatbot_dict):
    new_chatbot_dict = dict()
    for key in chatbot_dict.keys():
        new_chatbot_dict[key] = revive_chatbot(chatbot_dict[key])

    print(f'New chatbot dict keys: {[key for key in new_chatbot_dict]}')
    return new_chatbot_dict

new_chatbot_dict = revive_chatbot_dict(chatbot_dict)

Article title: Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1, 2]
	New chatbot attribute added: relevance_dict
		Attribute dictionary keys: [1, 2]
Article title: Comparisons in the Recovery Response From Resistance Ex

In [81]:
vars(new_chatbot_dict['text1_prompt00'])

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

## Iteration 2.1: Allow for select texts to be chosen only
Next step: When running revive_chatbot, add new attribute indicating number of previous prompts sent to ChatGPT for a given task type.

In [86]:
def revive_chatbot(chatbot):
    """
    Convert the dictionary of a chatbot attributes into a Chaining object.
    """
    new_chatbot = Chaining(chatbot['text'])
    print(f'Article title: {chatbot["article_title"]}')
    for key in chatbot.keys():
        # print(f'\t{chatbot[key]}')
        setattr(new_chatbot, key, chatbot[key])
        print(f'\tNew chatbot attribute added: {key}')
        if type(getattr(new_chatbot, key)) == dict:
            print(f'\t\tAttribute dictionary keys: {[attr for attr in getattr(new_chatbot, key)]}')
    return new_chatbot

def revive_chatbot_dict(chatbot_dict, texts='all'):
    """
    Convert the dictionary of dictionaries of chatbot attributes into a dictionary of Chaining objects.

    Parameters:
        - chatbot_dict (dict): dictionary of dictionaries of chatbot attributes.
        - texts (list or 'all'): list of integers corresponding to the text prompts to revive. 
            If 'all', all text prompts will be revived.
    """
    if texts == 'all':
        text_prompts_to_revive = [text for text in chatbot_dict.keys()]
    elif isinstance(texts, list):
        text_prompts_to_revive = [
            text_prompt for text_prompt in chatbot_dict.keys() for text in texts if f'{text}_' in text_prompt]
    else:
        raise TypeError("The `texts` parameter must be 'all' or a list of integers")

    new_chatbot_dict = {text_prompt: revive_chatbot(chatbot_dict[text_prompt]) for text_prompt in text_prompts_to_revive}
    print(f'New chatbot dict keys: {[key for key in new_chatbot_dict]}')
    return new_chatbot_dict

new_chatbot_dict = revive_chatbot_dict(chatbot_dict, texts=[1])

Article title: Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1, 2]
	New chatbot attribute added: relevance_dict
		Attribute dictionary keys: [1, 2]
Article title: Comparisons in the Recovery Response From Resistance Ex

# Send new add relevance tasks to previously generated summaries

In [90]:
user_relevance_task

['Update the summary with 1-2 sentences to make it relevant for']

In [94]:
relevance_prompts = user_relevance_task
relevance_dict = dict()
chaining_dict = dict()
chaining_dict[1] = new_chatbot_dict
pause_per_request = 0

iteration_id = 1
summary_iteration_id = iteration_id
relevance = prompt_chaining_dict(relevance_prompts, relevance_audience, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id, prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 1
n_choices: 1
**text1_prompt00
relevance_iteration:  3
Task: Update the summary with 1-2 sentences to make it relevant for seniors
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending re

In [99]:
vars(chaining_dict[1]['text1_prompt00'])

{'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during 

In [107]:
chaining_dict[1]['text1_prompt00'].relevance_dict[3]

{'response_01': {0: {'relevance choice': 1,
   'relevance task': 'Update the summary with 1-2 sentences to make it relevant for',
   'audience': 'seniors',
   'full relevance task': 'Update the summary with 1-2 sentences to make it relevant for seniors',
   'relevance statement': 'The study suggests that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individuals. However, changes in muscle performance may begin during middle age. Seniors who engage in resistance training should consider incorporating multijoint, dynamic constant resistance exercises to maintain muscle performance.',
   'preceding summary': 'A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to young

## Save API response

In [137]:
description = 'batch_Chaining_attributes_relevance_2023-04-19_1437_updated'
save_instance_to_dict(
    chaining_dict[1],
    description=description,
    save_json=False
)

text1_prompt00
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
text1_prompt01
	text
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/batch_Chaining_attributes_relevance_2023-04-19_1437_updated_2023-05-01_1928.sav
Time completed: 2023-05-01 19:28:12.602276
	Object saved as pickle
Dictionary keys: dict_keys(['text1_prompt00', 'text1_prompt01'])


{'text1_prompt00': {'text': 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength 

In [142]:
chaining_dict[1] = loadpickle('batch_Chaining_attributes_relevance_2023-04-19_1437_updated_2023-05-01_1928.sav', pickle_path)

Time completed: 2023-05-01 20:21:10.354730
Dictionary keys: ['text1_prompt00', 'text1_prompt01']


In [145]:
chaining_dict[1] = revive_chatbot_dict(chaining_dict[1])

Article title: Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1, 2]
	New chatbot attribute added: relevance_dict
		Attribute dictionary keys: [1, 2, 3, 4]
Article title: Comparisons in the Recovery Response From Resista

In [149]:
chaining_dict[1]

{'text1_prompt00': <__main__.Chaining at 0x231425e17f0>,
 'text1_prompt01': <__main__.Chaining at 0x23147421d60>}

In [150]:
chaining_dict[1]['text1_prompt00'].relevance_dict[3]

{'response_01': {0: {'relevance choice': 1,
   'relevance task': 'Update the summary with 1-2 sentences to make it relevant for',
   'audience': 'seniors',
   'full relevance task': 'Update the summary with 1-2 sentences to make it relevant for seniors',
   'relevance statement': 'The study suggests that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individuals. However, changes in muscle performance may begin during middle age. Seniors who engage in resistance training should consider incorporating multijoint, dynamic constant resistance exercises to maintain muscle performance.',
   'preceding summary': 'A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to young

In [216]:
print(len(chaining_dict[1]['text1_prompt00'].relevance_dict))
print(chaining_dict[1]['text1_prompt00'].relevance_dict.keys())
chaining_dict[1]['text1_prompt00'].relevance_dict[3]

4
dict_keys([1, 2, 3, 4])


{'response_01': {0: {'relevance choice': 1,
   'relevance task': 'Update the summary with 1-2 sentences to make it relevant for',
   'audience': 'seniors',
   'full relevance task': 'Update the summary with 1-2 sentences to make it relevant for seniors',
   'relevance statement': 'The study suggests that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to younger individuals. However, changes in muscle performance may begin during middle age. Seniors who engage in resistance training should consider incorporating multijoint, dynamic constant resistance exercises to maintain muscle performance.',
   'preceding summary': 'A study found no differences in recovery response between young and middle-aged men after high-volume resistance exercise, suggesting that middle-aged individuals who regularly engage in resistance training do not have an increased risk for muscle soreness or damage compared to young

In [170]:
chaining_dict[1]['text1_prompt00'].simple_summary_dict

{1: {'response_01': {0: {'simple summary choice': 1,
    'simplify task': 'Use language appropriate for',
    'audience': 'a lay audience',
    'full simplify task': 'Use language appropriate for for a lay audience',
    'simple summary': "A recent study found that middle-aged men who regularly do resistance training don't have a higher risk of muscle soreness or damage compared to younger men. This means that if you're middle-aged and you work out regularly, you don't have to worry about hurting your muscles more than younger people do. However, the study did find that muscle performance may start to change as you get older. The researchers suggest doing more exercises that use multiple joints and constant resistance to compare different age groups. So, if you're middle-aged and want to stay fit, keep up with your resistance training and try different exercises to keep your muscles strong.",
    'original summary': 'A study found no differences in recovery response between young and m

# Load previous qna_dict

In [125]:
filename = 'prompt_chain_simple_summaries_relevance_2023-04-19_1437.sav'
qna_df = loadpickle(filename, pickle_path)
qna_df

Time completed: 2023-05-01 19:18:35.893858
Dataframe shape:  (40, 14)
DataFrame columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice,audience,relevance task,full relevance task,relevance statement
0,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,For seniors who regularly engage in resistance...
1,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, this study highlights the importa..."
2,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,This study is particularly relevant for senior...
3,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,"""New study finds no difference in recovery res...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, resistance training can be an eff..."
4,Comparisons in the Recovery Response From Resi...,5,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in the rec...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, it is important to note that part..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,Recovery From Eccentric Squat Exercise in Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study investig...",1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for pe...,This study provides valuable insights for athl...
36,Recovery From Eccentric Squat Exercise in Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study looked a...",1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for pe...,This study is particularly relevant for athlet...
37,Recovery From Eccentric Squat Exercise in Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Recovery From Eccentric Squat Exercise in Resi...,Take the key points and numerical desc

In [126]:
filter = {'article_title': ['Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men']}

qna1 = filter_df_all_conditions(qna_df, filter)
qna1

Filtered on article_title in ['comparisons in the recovery response from resistance exercise between young and middle-aged men']
Results where ALL conditions are met:
	DataFrame indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
	DataFrame shape: (20, 14)


,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice,audience,relevance task,full relevance task,relevance statement
0,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,For seniors who regularly engage in resistance...
1,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, this study highlights the importa..."
2,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,This study is particularly relevant for senior...
3,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,"""New study finds no difference in recovery res...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, resistance training can be an eff..."
4,Comparisons in the Recovery Response From Resi...,5,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in the rec...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, it is important to note that part..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study found th...",1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for pe...,This study is great news for sports enthusiast...
16,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,Hey! I just read a study on the recovery respo...,1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for pe...,"For athletes and fitness enthusiasts, this stu..."
17,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical desc

In [127]:
qna_dict = dict()
qna_dict[1] = qna1

In [148]:
relevance[1]

{'text1_prompt00': <__main__.Chaining at 0x231425e17f0>,
 'text1_prompt01': <__main__.Chaining at 0x23147421d60>}

# Join new results with previous original summaries

In [168]:
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    print(f'New results columns: {new_results.columns}')
    # new_results = new_results[columns]

    if empty_columns:
        if (type(empty_columns) != dict):
            empty_columns = {
                'original summary content rating': 'k',
                'original summary language rating': 'l',
                'top summary': 'm',
            }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id,
    results_type='relevance'
)

New results shape: (40, 6)
New results columns: Index(['article_title', 'choice', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary',
       'relevance choice_x', 'audience_x', 'relevance task_x',
       'full relevance task_x', 'relevance statement_x', 'audience_y',
       'full relevance task_y', 'preceding summary', 'relevance choice_y',
       'relevance statement_y', 'relevance task_y'],
      dtype='object')
** added relevance dataframe shape: (80, 20)
['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice_x', 'audience_x', 'relevance task_x', 'full relevance task_x', 'relevance statement_x', 'audience_y', 'full relevance task_y', 'preceding summary', 'relevance choice_y', 'relevance statement_y', 'relevance task_y']


In [169]:
relevance_dict[1]

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice_x,audience_x,relevance task_x,full relevance task_x,relevance statement_x,audience_y,full relevance task_y,preceding summary,relevance choice_y,relevance statement_y,relevance task_y
0,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,For seniors who regularly engage in resistance...,seniors,Add 1-2 sentences to make this relevant for se...,A study found no differences in recovery respo...,1,For seniors who regularly engage in resistance...,Add 1-2 sentences to make this relevant for
1,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for pe...,This study is great news for middle-aged athle...,seniors,Add 1-2 sentences to make this relevant for se...,A study found no differences in recovery respo...,1,For seniors who regularly engage in resistance...,Add 1-2 sentences to make this relevant for
2,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, this study highlights the importa...",seniors,Add 1-2 sentences to make this relevant for se...,A recent study found no differences in recover...,1,"For seniors, this study highlights the importa...",Add 1-2 sentences to make this relevant for
3,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,1,people who enjoy sports,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for pe...,"For sports enthusiasts, this study highlights ...",seniors,Add 1-2 sentences to make this relevant for se...,A recent study found no differences in recover...,1,"For seniors, this study highlights the importa...",Add 1-2 sentences to make this relevant for
4,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,This study is particularly relevant for senior...,seniors,Add 1-2 sentences to make this relevant for se...,A study comparing young and middle-aged men in...,1,This study is particularly relevant for senior...,Add 1-2 sentences to make this relevant for
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Comparisons in the Recovery Response From Resi...,3,You are an expert at

## 2.1

In [175]:
import string
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    new_results = qna_dict[iteration_id][original_summary_columns].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    print(f'New results columns: {new_results.columns}')
    new_results = new_results[columns]

    if empty_columns:
        if (type(empty_columns) != dict):
            if results_type=='simple':
                empty_columns = {                    
                    "date added": "A",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                }
            else:
                empty_columns = {
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                    "added relevance content rating": "Y",
                    "added relevance language rating": "Z",
                    "top added relevance statement": "AA",
                }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            # print(f'{column_name}, {column_number} empty_column_loc: {empty_column_loc}')
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id,
    results_type='relevance', empty_columns=1
)

New results shape: (40, 6)
New results columns: Index(['article_title', 'choice', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary', 'audience',
       'full relevance task', 'preceding summary', 'relevance choice',
       'relevance statement', 'relevance task'],
      dtype='object')

Columns before adding empty columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance choice', 'audience', 'relevance task', 'full relevance task', 'relevance statement']
Inserting empty columns...
	K (9): original summary content rating, L (10): original summary language rating, M (11): top summary, N (12): simple summary choice, O (13): simplify audience, P (14): simplify task, Q (15): full simplify task, R (16): simple summary, S (17): simple summary content rating, T (18): simple summary language rating, U (19): top simple summary, Y (23): added 

In [166]:
relevance_dict[iteration_id]

,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: simplify audience,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: relevance choice,W: audience,X: relevance task,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance statement,AB: full relevance task,AC: relevance statement
0,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,,,,,,,,,,,,1,seniors,Add 1-2 sentences to make this relevant for,,,,Add 1-2 sentences to make this relevant for se...,For seniors who regularly engage in resistance...
1,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,,,,,,,,,,,,1,seniors,Add 1-2 sentences to make this relevant for,,,,Add 1-2 sentences to make this relevant for se...,For seniors who regularly engage in resistance...
2,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,,,,,,,,,,,,1,seniors,Add 1-2 sentences to make this relevant for,,,,Add 1-2 sentences to make this relevant for se...,"For seniors, this study highlights the importa..."
3,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,,,,,,,,,,,,1,seniors,Add 1-2 sentences to make this relevant for,,,,Add 1-2 sentences to make this relevant for se...,"For seniors, this study highlights the importa..."
4,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,,,,,,,,,,,,1,seniors,Add 1-2 sentences to make this relevant for,,,,Add 1-2 sentences to make this relevant for se...,This study is particularly relevant for senior...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study found th...",,,,,,,,,,,,1,people who enjoy sports,Update the summary with 1-2 sentences to make ...,,,,Update the summary with 1-2 sentences to make ...,"Hey sports enthusiasts, did you know that a re..."
76,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take

In [177]:
relevance_dict[iteration_id].filter(regex='full relevance task').value_counts()

AB: full relevance task                                                              
Add 1-2 sentences to make this relevant for people who enjoy sports                      20
Add 1-2 sentences to make this relevant for seniors                                      20
Update the summary with 1-2 sentences to make it relevant for people who enjoy sports    20
Update the summary with 1-2 sentences to make it relevant for seniors                    20
dtype: int64

In [198]:
sorted([key for key in chaining_dict[iteration_id].keys()])

['text1_prompt00', 'text1_prompt01']

## 2.2 Exclude previous relevance summaries

In [189]:
for key in chaining_dict[1]:
    chaining_dict[1][key].previous_n_prompts = {'relevance': 2}

In [194]:
for key in chaining_dict[1]:
    print(chaining_dict[1][key].previous_n_prompts)

{'relevance': 2}
{'relevance': 2}


In [210]:
import string
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None,
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'Number of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])[n_previous_prompts:]
            print(f'Appending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    new_results = qna_dict[iteration_id][original_summary_columns].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    print(f'New results columns: {new_results.columns}')
    new_results = new_results[columns]

    if empty_columns:
        if (type(empty_columns) != dict):
            if results_type=='simple':
                empty_columns = {                    
                    "date added": "A",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                }
            else:
                empty_columns = {
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                    "added relevance content rating": "Y",
                    "added relevance language rating": "Z",
                    "top added relevance statement": "AA",
                }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            # print(f'{column_name}, {column_number} empty_column_loc: {empty_column_loc}')
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


# relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id,
    results_type='relevance'#, empty_columns=1
)

Number of previous relevance prompts: 2
Appending results for 1: response_03, response_04, response_05, 
Appending results for 2: response_03, response_04, response_05, 
Appending results for 3: response_03, response_04, response_05, 
Appending results for 4: response_03, response_04, response_05, 
Number of previous relevance prompts: 2
Appending results for 1: response_03, response_04, response_05, 
Appending results for 2: response_03, response_04, response_05, 
Appending results for 3: response_03, response_04, response_05, 
Appending results for 4: response_03, response_04, response_05, 
New results shape: (24, 6)
New results columns: Index(['article_title', 'choice', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary', 'audience',
       'full relevance task', 'preceding summary', 'relevance choice',
       'relevance statement', 'relevance task'],
      dtype='object')
** added relevance dataframe shape: (48, 14)
['artic

In [207]:
relevance_df[iteration_id]

,article_title,choice,system_role,model,text,prep step,summarization task,full summarization task,summary,relevance choice,audience,relevance task,full relevance task,relevance statement
0,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,This study is particularly relevant for senior...
1,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,This study is particularly relevant for senior...
2,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,"""New study finds no difference in recovery res...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, resistance training can be an eff..."
3,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,"""New study finds no difference in recovery res...",1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, resistance training can be an eff..."
4,Comparisons in the Recovery Response From Resi...,5,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in the rec...,1,seniors,Add 1-2 sentences to make this relevant for,Add 1-2 sentences to make this relevant for se...,"For seniors, it is important to note that part..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study found th...",1,people who enjoy sports,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,"Hey sports enthusiasts, did you know that a re..."
44,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study found th...",1,people who enjoy sports,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,"For sports enthusiasts, this study shows that ..."
45,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points

In [205]:
relevance_dict[iteration_id].filter(regex='full relevance task').value_counts()

AB: full relevance task                                                              
Add 1-2 sentences to make this relevant for people who enjoy sports                      12
Add 1-2 sentences to make this relevant for seniors                                      12
Update the summary with 1-2 sentences to make it relevant for people who enjoy sports    12
Update the summary with 1-2 sentences to make it relevant for seniors                    12
dtype: int64

In [209]:
relevance_dict[iteration_id].filter(regex='article_title').value_counts()

article_title                                                                                  
Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men    48
dtype: int64

## 2.3 got it

In [ ]:
import string
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None, pivot=True:
        chatbot_id=None, save_df=False, save_chatbot=False,
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'\tNumber of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
        if results_type=='simple':
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
            chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number]
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])
            print(f'\tAppending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    new_results = qna_dict[iteration_id][original_summary_columns].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    if results_type=='simple':
        columns = [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'simple summary choice',
            'audience',
            'simplify task',
            'full simplify task',
            'simple summary'
        ]
    else:
        columns= [
            'article_title',
            'choice',
            'system_role',
            'model',
            'text',
            'prep step',
            'summarization task',
            'full summarization task',
            'summary',
            'relevance choice',
            'audience',
            'relevance task',
            'full relevance task',
            'relevance statement'
        ]
    print(f'New results columns: {new_results.columns}')
    new_results = new_results[columns]

    if empty_columns:
        if (type(empty_columns) != dict):
            if results_type=='simple':
                empty_columns = {                    
                    "date added": "A",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                }
            else:
                empty_columns = {
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            # print(f'{column_name}, {column_number} empty_column_loc: {empty_column_loc}')
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_simple_summaries_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


# relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id,
    results_type='relevance', empty_columns=1
)

Processing text1_prompt00...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
Processing text1_prompt01...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
New results shape: (20, 6)
New results columns: Index(['article_title', 'choice', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary', 'audience',
       'full relevance task', 'preceding summary', 'relevance choice',
       'relevance statement', 'relevance task'],
      dtype='object')

Columns before adding empty columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summa

In [235]:
relevance_df[iteration_id]

,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: relevance choice,O: simplify audience,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: audience,V: relevance task,W: full relevance task,X: relevance statement
0,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,,,,1,,,,,,,seniors,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,The study suggests that middle-aged individual...
1,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,,,,1,,,,,,,seniors,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,The study suggests that middle-aged individual...
2,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,,,,1,,,,,,,seniors,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,The study found that recreational resistance t...
3,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,,,,1,,,,,,,seniors,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,The study found that recreational resistance t...
4,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,,,,1,,,,,,,seniors,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,This study suggests that middle-aged adults ca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study found th...",,,,1,,,,,,,people who enjoy sports,Update the summary with 1-2 sentences to make ...,Update the summary with 1-2 sentences to make ...,"Hey sports enthusiasts, did you know that a re..."
36,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recen

In [227]:
relevance_dict[iteration_id].filter(regex='full relevance task').value_counts()

full relevance task                                                                  
Update the summary with 1-2 sentences to make it relevant for people who enjoy sports    20
Update the summary with 1-2 sentences to make it relevant for seniors                    20
dtype: int64

## 2.4 Pivot relevance results

In [286]:
import string
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, original_summmary_time='',
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'\tNumber of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
        if results_type=='simple':
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
            chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number]
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])
            print(f'\tAppending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    if (pivot == False) or (results_type=='simple'):
        new_results = qna_dict[iteration_id][original_summary_columns].merge(
            new_results, how='right', 
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
            left_on='summary'
        )
        if results_type=='simple':
            columns = [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'simple summary choice',
                'audience',
                'simplify task',
                'full simplify task',
                'simple summary'
            ]
        else:
            columns= [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'relevance choice',
                'audience',
                'relevance task',
                'full relevance task',
                'relevance statement'
            ]
        print(f'New results columns: {new_results.columns}')
        new_results = new_results[columns]
    else:
        new_results_pivot_df = new_results.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = qna_dict[iteration_id][original_summary_columns].merge(
            new_results_pivot_df, how='outer', suffixes=(' original', ' relevance'),
            left_on='summary',
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary',
            validate='m:1' if validate else None
        ).drop(columns=['preceding summary'])

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {                    
                        "date added": "A",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        # "simple summary choice": "N",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                        # "added relevance content rating": "Y",
                        # "added relevance language rating": "Z",
                        # "top added relevance statement": "AA",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'{original_summmary_time}_added_results_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


# relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id, 
    results_type='relevance', empty_columns=1
)

Processing text1_prompt00...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
Processing text1_prompt01...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
New results shape: (20, 6)

Columns before adding empty columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance task', 'people who enjoy sports', 'seniors']
Inserting empty columns...
	K (9): original summary content rating, L (10): original summary language rating, M (11): top summary, N (12): simple summary choice, O (13): simplify audience, P (14): simplify task, Q (15): full simplify task

In [284]:
relevance_df[iteration_id]

,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: simplify audience,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: relevance task,W: full add relevance task,X: people who enjoy sports,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance,AB: add relevance task (seniors),AC: full add relevance task (seniors),AD: seniors
0,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,,,,,,,,,,,,Update the summary with 1-2 sentences to make ...,,"For sports enthusiasts, this study suggests th...",,,,,,The study suggests that middle-aged individual...
1,Comparisons in the Recovery Response From Resi...,1,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study found no differences in recovery respo...,,,,,,,,,,,,Update the summary with 1-2 sentences to make ...,,"For sports enthusiasts, this study suggests th...",,,,,,The study suggests that middle-aged individual...
2,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,,,,,,,,,,,,Update the summary with 1-2 sentences to make ...,,"For sports enthusiasts, a recent study found t...",,,,,,The study found that recreational resistance t...
3,Comparisons in the Recovery Response From Resi...,2,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A recent study found no differences in recover...,,,,,,,,,,,,Update the summary with 1-2 sentences to make ...,,"For sports enthusiasts, a recent study found t...",,,,,,The study found that recreational resistance t...
4,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors ...,A study comparing young and middle-aged men in...,,,,,,,,,,,,Update the summary with 1-2 sentences to make ...,,This study shows that middle-aged adults can r...,,,,,,This study suggests that middle-aged adults ca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,Comparisons in the Recovery Response From Resi...,3,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery Response From Resi...,Take the key points and numerical descriptors to,Describe the interesting points to your cowork...,Take the key points and numerical descriptors ...,"Hey, did you know that a recent study found th...",,,,,,,,,,,,Update the summary with 1-2 sentences to make ...,,"Hey sports enthusiasts, did you know that a re...",,,,,,This study found that middle-aged men may expe...
16,Comparisons in the Recovery Response From Resi...,4,You are an expert at science communication.,gpt-3.5-turbo,Comparisons in the Recovery

## 2.5 Update to grab time stamp of original summaries to save to filename

In [288]:
for key in chaining_dict[1]:
    chaining_dict[1][key].created_date = '2023-04-19_1437'

for key in chaining_dict[1]:
    print(chaining_dict[1][key].created_date)

2023-04-19_1437
2023-04-19_1437


In [289]:
next(iter(chaining_dict[1].values())).created_date

'2023-04-19_1437'

In [297]:
import string
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'\tNumber of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
        if results_type=='simple':
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
            chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number]
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])
            print(f'\tAppending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    if (pivot == False) or (results_type=='simple'):
        new_results = qna_dict[iteration_id][original_summary_columns].merge(
            new_results, how='right', 
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
            left_on='summary'
        )
        if results_type=='simple':
            columns = [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'simple summary choice',
                'audience',
                'simplify task',
                'full simplify task',
                'simple summary'
            ]
        else:
            columns= [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'relevance choice',
                'audience',
                'relevance task',
                'full relevance task',
                'relevance statement'
            ]
        print(f'New results columns: {new_results.columns}')
        new_results = new_results[columns]
    else:
        new_results_pivot_df = new_results.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = qna_dict[iteration_id][original_summary_columns].merge(
            new_results_pivot_df, how='outer', suffixes=(' original', ' relevance'),
            left_on='summary',
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary',
            validate='m:1' if validate else None
        ).drop(columns=['preceding summary'])

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {                    
                        "date added": "A",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        # "simple summary choice": "N",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                        # "added relevance content rating": "Y",
                        # "added relevance language rating": "Z",
                        # "top added relevance statement": "AA",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).created_date
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_summaries_and_relevance_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


# relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id, save_df=True, save_chatbot=True,
    results_type='relevance', empty_columns=1
)

Processing text1_prompt00...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
Processing text1_prompt01...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
New results shape: (20, 6)

Columns before adding empty columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance task', 'people who enjoy sports', 'seniors']
Inserting empty columns...
	K (9): original summary content rating, L (10): original summary language rating, M (11): top summary, N (12): simple summary choice, O (13): simplify audience, P (14): simplify task, Q (15): full simplify task

## 2.6 change attribute name from `created_date` to `date_created`

In [ ]:
import string
def process_chaining_results2(
        chain_results_dict, qna_dict, chatbot_dict, iteration_id, results_type='simple',
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts[results_type]
            print(f'\tNumber of previous {results_type} prompts: {n_previous_prompts}')
        except:
            n_previous_prompts = 0
            print(f'No previous {results_type} prompts for {chatbot_key}')
        if results_type=='simple':
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
            chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            total_n_prompts = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
            results_dict = dict()
            for prompt_number in range(n_previous_prompts+1, total_n_prompts+1):
                results_dict[prompt_number] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number]
        for iteration_key in results_dict.keys():
            response_keys = sorted([text_prompt_key for text_prompt_key in results_dict[iteration_key].keys()])
            print(f'\tAppending results for {iteration_key}: ', end='')
            for response_key in response_keys:
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
                print(f'{response_key}, ', end='')
            print('')

    
    new_results = pd.concat(df_list)
    print('New results shape:', new_results.shape)
    
    original_summary_columns = [
        'article_title',
        'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    if (pivot == False) or (results_type=='simple'):
        new_results = qna_dict[iteration_id][original_summary_columns].merge(
            new_results, how='right', 
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
            left_on='summary'
        )
        if results_type=='simple':
            columns = [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'simple summary choice',
                'audience',
                'simplify task',
                'full simplify task',
                'simple summary'
            ]
        else:
            columns= [
                'article_title',
                'choice',
                'system_role',
                'model',
                'text',
                'prep step',
                'summarization task',
                'full summarization task',
                'summary',
                'relevance choice',
                'audience',
                'relevance task',
                'full relevance task',
                'relevance statement'
            ]
        print(f'New results columns: {new_results.columns}')
        new_results = new_results[columns]
    else:
        new_results_pivot_df = new_results.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = qna_dict[iteration_id][original_summary_columns].merge(
            new_results_pivot_df, how='outer', suffixes=(' original', ' relevance'),
            left_on='summary',
            right_on=f'{"original" if results_type=="simple" else "preceding"} summary',
            validate='m:1' if validate else None
        ).drop(columns=['preceding summary'])

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {                    
                        "date added": "A",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        # "simple summary choice": "N",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** {"simple summaries" if results_type=="simple" else "added relevance"} dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'prompt_chain_summaries_and_relevance_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict


# relevance_dict = dict()
relevance_df = process_chaining_results2(
    relevance_dict, qna_dict, chaining_dict,
    iteration_id=iteration_id, save_df=True, save_chatbot=True,
    results_type='relevance', empty_columns=1
)

Processing text1_prompt00...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
Processing text1_prompt01...
	Number of previous relevance prompts: 2
	Appending results for 3: response_01, response_02, response_03, response_04, response_05, 
	Appending results for 4: response_01, response_02, response_03, response_04, response_05, 
New results shape: (20, 6)

Columns before adding empty columns: ['article_title', 'choice', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary', 'relevance task', 'people who enjoy sports', 'seniors']
Inserting empty columns...
	K (9): original summary content rating, L (10): original summary language rating, M (11): top summary, N (12): simple summary choice, O (13): simplify audience, P (14): simplify task, Q (15): full simplify task

# *End of Page*